In [24]:
import os
import glob
import pandas as pd
from tqdm import tqdm

In [25]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Imports

In [26]:
# Brian's data
# df = pd.read_csv("../mimic-iv-3.1/ecoli_targets_w_lab_features.csv.gz", compression="gzip")
df = pd.read_csv("../mimic-iv-3.1/targets_w_lab_features.csv.gz", compression="gzip")

C:\Users\brian\AppData\Local\Temp\ipykernel_4432\3114246561.py:3: DtypeWarning: Columns (11,12,13,14,17,20,23,36,43,45,46,47,48,58,60,61,66,67,69,70,71,75,78,83,84,90,95,96,98,99,103,105) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../mimic-iv-3.1/targets_w_lab_features.csv.gz", compression="gzip")


In [27]:
# Kat's data
omr_df = pd.read_csv("../mimic-iv-3.1/df_omr_pt_ad_diag_pr.csv")

In [28]:
print(df.shape)
print(omr_df.shape)

(504738, 108)
(35529, 928)


### Merge

In [29]:
df = df.merge(omr_df, 
              on=["subject_id","charttime"], 
              how="left")

In [30]:
df.shape

(504738, 1034)

In [31]:
# Count NaN values in each of the new lab test columns
nan_counts = df.isna().sum()

# Display the result
print(nan_counts)
print(df.shape[0])

subject_id                                      0
hadm_id                                    321967
org_name                                        0
ab_name                                         0
charttime                                       0
interpretation                                  0
susceptible_flag                                0
charttime_target                                0
Blood - Hematocrit                          41097
Blood - Platelet Count                      42433
Blood - MCV                                 41691
Blood - Red Blood Cells                     41691
Blood - RDW                                 41846
Blood - MCH                                 41742
Blood - MCHC                                41729
Blood - White Blood Cells                   41775
Blood - Creatinine                          44755
Blood - Urea Nitrogen                       46507
Blood - Potassium                           51471
Blood - Sodium                              51553


In [32]:
df.dtypes

subject_id                                   int64
hadm_id                                    float64
org_name                                    object
ab_name                                     object
charttime                                   object
interpretation                              object
susceptible_flag                           float64
charttime_target                            object
Blood - Hematocrit                          object
Blood - Platelet Count                      object
Blood - MCV                                 object
Blood - Red Blood Cells                     object
Blood - RDW                                 object
Blood - MCH                                 object
Blood - MCHC                                object
Blood - White Blood Cells                   object
Blood - Creatinine                          object
Blood - Urea Nitrogen                       object
Blood - Potassium                           object
Blood - Sodium                 

In [33]:
df.drop(columns=['subject_id','hadm_id','interpretation','charttime_target'], inplace=True)
df.head()

,org_name,ab_name,charttime,susceptible_flag,Blood - Hematocrit,Blood - Platelet Count,Blood - MCV,Blood - Red Blood Cells,Blood - RDW,Blood - MCH,Blood - MCHC,Blood - White Blood Cells,Blood - Creatinine,Blood - Urea Nitrogen,Blood - Potassium,Blood - Sodium,Blood - Chloride,Blood - Bicarbonate,Blood - Anion Gap,Blood - Hemoglobin,Blood - Glucose,Urine - Urine Color,Urine - Urine Appearance,Urine - Urobilinogen,Urine - Specific Gravity,Urine - Leukocytes,Urine - Ketone,Urine - Blood,Urine - Glucose,Urine - Protein,Urine - Bilirubin,Urine - Nitrite,Urine - pH,Urine - RBC,Urine - WBC,Urine - Bacteria,Urine - Yeast,Urine - Epithelial Cells,Blood - Estimated GFR (MDRD equation),Blood - Monocytes,Blood - Neutrophils,Blood - Basophils,Blood - Lymphocytes,Blood - Eosinophils,"Blood - Calcium, Total",Blood - Alanine Aminotransferase (ALT),Blood - Asparate Aminotransferase (AST),Blood - Magnesium,Blood - Phosphate,Blood - INR(PT),Blood - PT,Blood - Alkaline Phosphatase,Blood - PTT,Urine - Length of Urine Collection,"Blood - Bilirubin, Total",Urine - Urine Mucous,Blood - Albumin,Blood - RDW-SD,Blood - Lactate,Blood - H,Blood - L,Blood - I,Blood - Absolute Lymphocyte Count,Blood - Absolute Neutrophil Count,Blood - Green Top Hold (plasma),Blood - Absolute Eosinophil Count,Blood - Absolute Basophil Count,Blood - Absolute Monocyte Count,Blood - Blue Top Hold,Blood - Immature Granulocytes,Blood - Light Green Top Hold,Blood - Thyroid Stimulating Hormone,Blood - Troponin T,Urine - Uhold,Blood - Lipase,Blood - Triglycerides,Blood - Red Top Hold,Blood - Creatine Kinase (CK),Blood - Specimen Type,"Blood - Cholesterol, Total","Blood - Cholesterol, HDL",Blood - Cholesterol Ratio (Total/HDL),Urine - Hyaline Casts,Blood - Lactate Dehydrogenase (LD),"Blood - Cholesterol, LDL, Calculated",Blood - % Hemoglobin A1c,Blood - Ferritin,Blood - eAG,"Blood - Potassium, Whole Blood",Blood - pH,Urine - Transitional Epithelial Cells,"Urine - Creatinine, Urine",Blood - Iron,"Urine - Urine tube, held",Blood - Vitamin B12,Blood - Base Excess,Blood - Calculated Total CO2,Blood - pO2,Blood - pCO2,Blood - Transferrin,"Blood - Iron Binding Capacity, Total","Blood - Creatine Kinase, MB Isoenzyme",Blood - Platelet Smear,Blood - Poikilocytosis,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,gender,anchor_age,insurance,language,marital_status,race,99,BLD001,BLD002,BLD003,BLD004,BLD005,BLD006,BLD007,BLD008,BLD009,BLD010,CIR001,CIR002,CIR003,CIR004,CIR005,CIR006,CIR007,CIR008,CIR009,CIR010,CIR011,CIR012,CIR013,CIR014,CIR015,CIR016,CIR017,CIR018,CIR019,CIR020,CIR021,CIR022,CIR023,CIR024,CIR025,CIR026,CIR027,CIR028,CIR029,CIR030,CIR031,CIR032,CIR033,CIR034,CIR035,CIR036,CIR037,CIR038,CIR039,DEN001,DEN002,DEN003,DIG001,DIG002,DIG003,DIG004,DIG005,DIG006,DIG007,DIG008,DIG009,DIG010,DIG011,DIG012,DIG013,DIG014,DIG015,DIG016,DIG017,DIG018,DIG019,DIG020,DIG021,DIG022,DIG023,DIG024,DIG025,EAR001,EAR002,EAR003,EAR004,EAR005,EAR006,END001,END002,END003,END004,END005,END006,END007,END008,END009,END010,END011,END012,END013,END014,END015,END016,END017,EXT001,EXT002,EXT003,EXT004,EXT005,EXT006,EXT007,EXT008,EXT009,EXT010,EXT011,EXT012,EXT013,EXT014,EXT015,EXT016,EXT017,EXT018,EXT019,EXT020,EXT021,EXT022,EXT023,EXT024,EXT025,EXT026,EXT027,EXT028,EXT029,EXT030,EYE001,EYE002,EYE003,EYE004,EYE005,EYE006,EYE007,EYE008,EYE009,EYE010,EYE011,EYE012,FAC001,FAC002,FAC003,FAC004,FAC005,FAC006,FAC008,FAC009,FAC010,FAC012,FAC013,FAC014,FAC015,FAC016,FAC017,FAC019,FAC020,FAC022,FAC023,FAC024,FAC025,FAC026,FAC027,FAC028,FAC029,FAC030,GEN001,GEN002,GEN003,GEN004,GEN005,GEN006,GEN007,GEN008,GEN009,GEN010,GEN011,GEN012,GEN013,GEN014,GEN015,GEN016,GEN017,GEN018,GEN019,GEN020,GEN021,GEN022,GEN023,GEN024,GEN025,GEN026,INF001,INF002,INF003,INF004,INF005,INF006,INF007,INF008,INF009,INF010,INF011,INF012,INJ001,INJ002,INJ003,INJ004,INJ005,INJ006,INJ007,INJ008,INJ009,INJ010,INJ011,INJ012,INJ013,INJ014,INJ015,INJ016,INJ017,INJ018,INJ019,INJ020,INJ021,INJ022,INJ023,INJ024,INJ025,INJ026,INJ027,INJ028,INJ029,INJ

### Feature Selection

In [34]:
target = ['susceptible_flag']

numeric_features = [
    "Blood - Hematocrit",
    "Blood - Platelet Count",
    "Blood - MCV",
    "Blood - Red Blood Cells",
    "Blood - RDW",
    "Blood - MCH",
    "Blood - MCHC",
    "Blood - White Blood Cells",
    "Blood - Creatinine",
    "Blood - Urea Nitrogen",
#    "Blood - Potassium",
#    "Blood - Sodium",
    "Blood - Chloride",
    "Blood - Bicarbonate",
#    "Blood - Anion Gap",
    "Blood - Hemoglobin",
    'anchor_age',
    'Height (Inches)',
    'Weight (Lbs)',
    'bp_systolic',
    'bp_diastolic'
]
cat_features = [
    'Urine - Nitrite',
    'Urine - Bilirubin',
    'Urine - Yeast',
    'gender',
    'insurance',
    'language',
    'marital_status',
    'race'    
]
OHE_features = [
    '99',
'BLD001','BLD002','BLD003','BLD004','BLD005','BLD006','BLD007','BLD008','BLD009','BLD010',
'CIR001','CIR002','CIR003','CIR004','CIR005','CIR006','CIR007','CIR008','CIR009','CIR010',
'CIR011','CIR012','CIR013','CIR014','CIR015','CIR016','CIR017','CIR018','CIR019','CIR020',
'CIR021','CIR022','CIR023','CIR024','CIR025','CIR026','CIR027','CIR028','CIR029','CIR030',
'CIR031','CIR032','CIR033','CIR034','CIR035','CIR036','CIR037','CIR038','CIR039',
'DEN001','DEN002','DEN003',
'DIG001','DIG002','DIG003','DIG004','DIG005','DIG006','DIG007','DIG008','DIG009','DIG010',
'DIG011','DIG012','DIG013','DIG014','DIG015','DIG016','DIG017','DIG018','DIG019','DIG020',
'DIG021','DIG022','DIG023','DIG024','DIG025',
'EAR001','EAR002','EAR003','EAR004','EAR005','EAR006',
'END001','END002','END003','END004','END005','END006','END007','END008','END009','END010',
'END011','END012','END013','END014','END015','END016','END017',
'EXT001','EXT002','EXT003','EXT004','EXT005','EXT006','EXT007','EXT008','EXT009','EXT010',
'EXT011','EXT012','EXT013','EXT014','EXT015','EXT016','EXT017','EXT018','EXT019','EXT020',
'EXT021','EXT022','EXT023','EXT024','EXT025','EXT026','EXT027','EXT028','EXT029','EXT030',
'EYE001','EYE002','EYE003','EYE004','EYE005','EYE006','EYE007','EYE008','EYE009','EYE010',
'EYE011','EYE012',
'FAC001','FAC002','FAC003','FAC004','FAC005','FAC006','FAC008','FAC009','FAC010',
'FAC012','FAC013','FAC014','FAC015','FAC016','FAC017','FAC019','FAC020', #FAC011 missing
'FAC022','FAC023','FAC024','FAC025','FAC026','FAC027','FAC028','FAC029','FAC030', #FAC021 missing
'GEN001','GEN002','GEN003','GEN004','GEN005','GEN006','GEN007','GEN008','GEN009','GEN010',
'GEN011','GEN012','GEN013','GEN014','GEN015','GEN016','GEN017','GEN018','GEN019','GEN020',
'GEN021','GEN022','GEN023','GEN024','GEN025','GEN026',
'INF001','INF002','INF003','INF004','INF005','INF006','INF007','INF008','INF009','INF010',
'INF011','INF012',
'INJ001','INJ002','INJ003','INJ004','INJ005','INJ006','INJ007','INJ008','INJ009','INJ010',
'INJ011','INJ012','INJ013','INJ014','INJ015','INJ016','INJ017','INJ018','INJ019','INJ020',
'INJ021','INJ022','INJ023','INJ024','INJ025','INJ026','INJ027','INJ028','INJ029','INJ030',
'INJ031','INJ032','INJ033','INJ034','INJ035','INJ036','INJ037','INJ038','INJ039','INJ040',
'INJ041','INJ042','INJ043','INJ044','INJ045','INJ046','INJ047','INJ048','INJ049','INJ050',
'INJ054','INJ055','INJ056','INJ057','INJ059','INJ060','INJ061','INJ062','INJ063','INJ064',
'INJ065','INJ066','INJ067','INJ069','INJ070','INJ071','INJ072','INJ073','INJ074','INJ075','INJ076',
'MAL001','MAL002','MAL003','MAL004','MAL005','MAL006','MAL007','MAL008','MAL009','MAL010',
'MBD001','MBD002','MBD003','MBD004','MBD005','MBD006','MBD007','MBD008','MBD009','MBD010',
'MBD011','MBD012','MBD013','MBD014','MBD017','MBD018','MBD019','MBD020','MBD021','MBD022',
'MBD023','MBD024','MBD025','MBD026','MBD027','MBD028','MBD032','MBD034',
'MUS001','MUS002','MUS003','MUS004','MUS005','MUS006','MUS007','MUS008','MUS009','MUS010',
'MUS011','MUS012','MUS013','MUS014','MUS015','MUS016','MUS017','MUS020','MUS021','MUS022',
'MUS023','MUS024','MUS025','MUS026','MUS027','MUS028','MUS029','MUS030','MUS031','MUS032',
'MUS033','MUS034','MUS035','MUS036','MUS037','MUS038',
'NEO001','NEO002','NEO003','NEO004','NEO005','NEO006','NEO007','NEO008','NEO009','NEO010',
'NEO011','NEO012','NEO013','NEO014','NEO015','NEO016','NEO017','NEO018','NEO019','NEO020',
'NEO021','NEO022','NEO023','NEO024','NEO025','NEO026','NEO027','NEO028','NEO029','NEO030',
'NEO031','NEO032','NEO033','NEO034','NEO035','NEO036','NEO037','NEO038','NEO039','NEO040',
'NEO041','NEO042','NEO043','NEO044','NEO045','NEO046','NEO047','NEO048','NEO049','NEO050',
'NEO051','NEO052','NEO054','NEO056','NEO057','NEO058','NEO059','NEO060','NEO061','NEO062',
'NEO063','NEO064','NEO065','NEO066','NEO067','NEO068','NEO069','NEO070','NEO071','NEO072',
'NEO073','NEO074','NEO075','NEO078',
'NVS001','NVS002','NVS003','NVS004','NVS005','NVS006','NVS007','NVS008','NVS009','NVS010',
'NVS011','NVS012','NVS013','NVS014','NVS015','NVS016','NVS017','NVS018','NVS019','NVS020',
'NVS021','NVS022',
'PNL004','PNL006','PNL009','PNL010','PNL013','PNL015',
'PRG002','PRG003','PRG004','PRG005','PRG006','PRG007','PRG008','PRG009','PRG010','PRG011',
'PRG012','PRG013','PRG014','PRG015','PRG016','PRG017','PRG018','PRG019','PRG020','PRG021',
'PRG022','PRG023','PRG024','PRG025','PRG026','PRG027','PRG028','PRG029','PRG030',
'RSP001','RSP002','RSP003','RSP004','RSP005','RSP006','RSP007','RSP008','RSP009','RSP010',
'RSP011','RSP012','RSP013','RSP014','RSP015','RSP016','RSP017',
'SKN001','SKN002','SKN003','SKN004','SKN005','SKN006','SKN007',
'SYM001','SYM002','SYM003','SYM004','SYM005','SYM006','SYM007','SYM008','SYM009','SYM010',
'SYM011','SYM012','SYM013','SYM014','SYM015','SYM016','SYM017','SYM018','99.1',
'A01AA','A01AB','A01AC','A01AD','A02AA','A02AB','A02AC','A02AF','A02BA','A02BB','A02BC',
'A02BX','A03AA','A03AB','A03BA','A03CB','A03FA','A04AA','A04AD','A05AA','A06AA','A06AB',
'A06AD','A06AG','A06AH','A06AX','A07AA','A07AC','A07BA','A07DA','A07EA','A07EB','A07EC',
'A09AA','A09AB','A10AB','A10AC','A10AD','A10AE','A10AF','A10BA','A10BB','A10BF','A10BH',
'A10BJ','A10BK','A10BX','A11CC','A12AA','A12BA','A12CA','A12CC','A12CD','A16AX',
'B01AA','B01AB','B01AC','B01AD','B01AE','B01AF','B02AA','B02BA','B02BC','B02BD','B02BX',
'B03AB','B03AC','B03BA','B03XA','B05AA','B05BA','B05BB','B05BC','B05CA','B05CB','B05CX',
'B05XA','B05XB','B05XX','B06AA','C01AA','C01BA','C01BB','C01BC','C01BD','C01CA','C01CE',
'C01DA','C01EA','C01EB','C02AC','C02CA','C02DB','C02DC','C03AA','C03BA','C03CA','C03DA',
'C03DB','C05AA','C05AD','C05AE','C05AX','C05BA','C05BB','C07AA','C07AB','C07AG','C08CA',
'C08DA','C08DB','C09AA','C09CA','C09DX','C10AA','C10AC','D01AA','D01AC','D01AE','D01BA',
'D02AA','D04AA','D04AB','D05AX','D06AX','D06BB','D06BX','D07AA','D07AB','D07AC','D07AD',
'D07XA','D07XB','D07XC','D08AE','D10AA','D10AB','D10AD','D10AE','D10AF','D11AA','D11AC',
'D11AH','D11AX','G01AA','G01AF','G02AB','G02AD','G02CB','G02CC','G03AA','G03AB','G03AC',
'G03BA','G03CA','G03DA','G03DB','G03DC','G03FA','G03FB','G03XA','G03XC','G04BA','G04BC',
'G04BD','G04BE','G04BX','G04CA','G04CB','H01CB','H02AA','H02AB','H03AA','H04AA','H05AA',
'H05BX','J01AA','J01CA','J01CE','J01CF','J01CG','J01CR','J01DB','J01DD','J01DE','J01DH',
'J01DI','J01EA','J01EC','J01EE','J01FA','J01FF','J01GB','J01MA','J01XA','J01XB','J01XD',
'J01XX','J02AA','J02AB','J02AC','J02AX','J04AB','J04AC','J04AD','J05AB','J05AC','J05AE',
'J05AF','J05AG','J05AH','J05AJ','J05AR','J05AX','J06BA','J07AH','J07AL','J07BC','J07BD',
'J07BK','L01AA','L01AC','L01AX','L01BA','L01BB','L01BC','L01CB','L01CD','L01CE','L01DB',
'L01EA','L01EC','L01EF','L01EG','L01EL','L01EX','L01FA','L01FF','L01FX','L01XA','L01XF',
'L01XK','L01XX','L02AA','L02AB','L02AE','L02BA','L02BB','L02BG','L03AB','L03AX','L04AA',
'L04AB','L04AC','L04AD','L04AF','L04AH','L04AK','L04AX','M01AB','M01AC','M01AE','M01AH',
'M01CC','M02AA','M02AB','M03AB','M03AC','M03AX','M03BX','M04AA','M04AB','M05BA','N01AH',
'N01AX','N01BA','N01BB','N01BX','N02AA','N02AB','N02AE','N02AF','N02AJ','N02BA','N02BE',
'N02BF','N02CA','N02CC','N02CX','N03AA','N03AB','N03AF','N03AG','N03AX','N04AC','N04BA',
'N04BB','N04BC','N04BX','N05AA','N05AB','N05AD','N05AE','N05AH','N05AN','N05AX','N05BA',
'N05BB','N05CD','N05CF','N05CH','N05CM','N06AA','N06AB','N06AF','N06AX','N06BA','N06BC',
'N06DA','N06DX','N07AA','N07AB','N07AX','N07BA','N07BB','N07BC','P01AB','P01AX','P01BA',
'P01BB','P02CF','P03AC','R01AA','R01AB','R01AC','R01AD','R01AX','R01BA','R02AA','R02AB',
'R02AD','R02AX','R03AA','R03AC','R03AK','R03AL','R03BA','R03BB','R03BC','R03CC','R03DC',
'R03DX','R05CA','R05CB','R05DA','R06AA','R06AB','R06AD','R06AE','S01AA','S01AD','S01AE',
'S01BA','S01BC','S01CB','S01EA','S01EB','S01EC','S01EE','S01FA','S01FB','S01GA','S01GX',
'S01HA','S01XA','S02AA','S02BA','S02DA','S03AA','S03BA','V03AB','V03AC','V03AE','V03AF',
'V04CA','V04CC','V04CG','V04CX','V06DC','V07AB'
]

In [35]:
selected_cols = target + numeric_features + cat_features + OHE_features
# Filter dataframe
df_filtered = df[selected_cols]

# Display the shape of the new dataframe
print(df_filtered.shape)

(504738, 943)


In [15]:
df_filtered.dropna().head()

,susceptible_flag,Blood - Hematocrit,Blood - Platelet Count,Blood - MCV,Blood - Red Blood Cells,Blood - RDW,Blood - MCH,Blood - MCHC,Blood - White Blood Cells,Blood - Creatinine,Blood - Urea Nitrogen,Blood - Chloride,Blood - Bicarbonate,Blood - Hemoglobin,anchor_age,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,Urine - Nitrite,Urine - Bilirubin,Urine - Yeast,gender,insurance,language,marital_status,race,99,BLD001,BLD002,BLD003,BLD004,BLD005,BLD006,BLD007,BLD008,BLD009,BLD010,CIR001,CIR002,CIR003,CIR004,CIR005,CIR006,CIR007,CIR008,CIR009,CIR010,CIR011,CIR012,CIR013,CIR014,CIR015,CIR016,CIR017,CIR018,CIR019,CIR020,CIR021,CIR022,CIR023,CIR024,CIR025,CIR026,CIR027,CIR028,CIR029,CIR030,CIR031,CIR032,CIR033,CIR034,CIR035,CIR036,CIR037,CIR038,CIR039,DEN001,DEN002,DEN003,DIG001,DIG002,DIG003,DIG004,DIG005,DIG006,DIG007,DIG008,DIG009,DIG010,DIG011,DIG012,DIG013,DIG014,DIG015,DIG016,DIG017,DIG018,DIG019,DIG020,DIG021,DIG022,DIG023,DIG024,DIG025,EAR001,EAR002,EAR003,EAR004,EAR005,EAR006,END001,END002,END003,END004,END005,END006,END007,END008,END009,END010,END011,END012,END013,END014,END015,END016,END017,EXT001,EXT002,EXT003,EXT004,EXT005,EXT006,EXT007,EXT008,EXT009,EXT010,EXT011,EXT012,EXT013,EXT014,EXT015,EXT016,EXT017,EXT018,EXT019,EXT020,EXT021,EXT022,EXT023,EXT024,EXT025,EXT026,EXT027,EXT028,EXT029,EXT030,EYE001,EYE002,EYE003,EYE004,EYE005,EYE006,EYE007,EYE008,EYE009,EYE010,EYE011,EYE012,FAC001,FAC002,FAC003,FAC004,FAC005,FAC006,FAC008,FAC009,FAC010,FAC012,FAC013,FAC014,FAC015,FAC016,FAC017,FAC019,FAC020,FAC022,FAC023,FAC024,FAC025,FAC026,FAC027,FAC028,FAC029,FAC030,GEN001,GEN002,GEN003,GEN004,GEN005,GEN006,GEN007,GEN008,GEN009,GEN010,GEN011,GEN012,GEN013,GEN014,GEN015,GEN016,GEN017,GEN018,GEN019,GEN020,GEN021,GEN022,GEN023,GEN024,GEN025,GEN026,INF001,INF002,INF003,INF004,INF005,INF006,INF007,INF008,INF009,INF010,INF011,INF012,INJ001,INJ002,INJ003,INJ004,INJ005,INJ006,INJ007,INJ008,INJ009,INJ010,INJ011,INJ012,INJ013,INJ014,INJ015,INJ016,INJ017,INJ018,INJ019,INJ020,INJ021,INJ022,INJ023,INJ024,INJ025,INJ026,INJ027,INJ028,INJ029,INJ030,INJ031,INJ032,INJ033,INJ034,INJ035,INJ036,INJ037,INJ038,INJ039,INJ040,INJ041,INJ042,INJ043,INJ044,INJ045,INJ046,INJ047,INJ048,INJ049,INJ050,INJ054,INJ055,INJ056,INJ057,INJ059,INJ060,INJ061,INJ062,INJ063,INJ064,INJ065,INJ066,INJ067,INJ069,INJ070,INJ071,INJ072,INJ073,INJ074,INJ075,INJ076,MAL001,MAL002,MAL003,MAL004,MAL005,MAL006,MAL007,MAL008,MAL009,MAL010,MBD001,MBD002,MBD003,MBD004,MBD005,MBD006,MBD007,MBD008,MBD009,MBD010,MBD011,MBD012,MBD013,MBD014,MBD017,MBD018,MBD019,MBD020,MBD021,MBD022,MBD023,MBD024,MBD025,MBD026,MBD027,MBD028,MBD032,MBD034,MUS001,MUS002,MUS003,MUS004,MUS005,MUS006,MUS007,MUS008,MUS009,MUS010,MUS011,MUS012,MUS013,MUS014,MUS015,MUS016,MUS017,MUS020,MUS021,MUS022,MUS023,MUS024,MUS025,MUS026,MUS027,MUS028,MUS029,MUS030,MUS031,MUS032,MUS033,MUS034,MUS035,MUS036,MUS037,MUS038,NEO001,NEO002,NEO003,NEO004,NEO005,NEO006,NEO007,NEO008,NEO009,NEO010,NEO011,NEO012,NEO013,NEO014,NEO015,NEO016,NEO017,NEO018,NEO019,NEO020,NEO021,NEO022,NEO023,NEO024,NEO025,NEO026,NEO027,NEO028,NEO029,NEO030,NEO031,NEO032,NEO033,NEO034,NEO035,NEO036,NEO037,NEO038,NEO039,NEO040,NEO041,NEO042,NEO043,NEO044,NEO045,NEO046,NEO047,NEO048,NEO049,NEO050,NEO051,NEO052,NEO054,NEO056,NEO057,NEO058,NEO059,NEO060,NEO061,NEO062,NEO063,NEO064,NEO065,NEO066,NEO067,NEO068,NEO069,NEO070,NEO071,NEO072,NEO073,NEO074,NEO075,NEO078,NVS001,NVS002,NVS003,NVS004,NVS005,NVS006,NVS007,NVS008,NVS009,NVS010,NVS011,NVS012,NVS013,NVS014,NVS015,NVS016,NVS017,NVS018,NVS019,NVS020,NVS021,NVS022,PNL004,PNL006,PNL009,PNL010,PNL013,PNL015,PRG002,PRG003,PRG004,PRG005,PRG006,PRG007,PRG008,PRG009,PRG010,PRG011,PRG012,PRG013,PRG014,PRG015,PRG016,PRG017,PRG018,PRG019,PRG020,PRG021,PRG022,PRG023,PRG024,PRG025,PRG026,PRG027,PRG028,PRG029,PRG030,RSP001,RSP002,RSP003,RSP004,RSP005,RSP006,RSP007,RSP008,RSP009,RSP010,RSP011,RSP012,RSP013,RSP014,RSP015,RSP016,RSP017,SKN001,SKN002,SKN003,SKN004,SKN005,SKN006,SKN007,SYM001,SYM002,SYM003,SYM004,SYM005,SYM00

### OHE

In [16]:
from sklearn.preprocessing import OneHotEncoder

# Initialize encoder
encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# Fit and transform categorical data
encoded_array = encoder.fit_transform(df_filtered[cat_features])

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out(cat_features))

# Concatenate with original dataframe (dropping old categorical columns)
df_final = pd.concat([df_filtered.drop(columns=cat_features), encoded_df], axis=1)

# Display new dataframe shape
print(df_final.shape)

# Check new columns
df_final.head()


(27359, 1019)


,susceptible_flag,Blood - Hematocrit,Blood - Platelet Count,Blood - MCV,Blood - Red Blood Cells,Blood - RDW,Blood - MCH,Blood - MCHC,Blood - White Blood Cells,Blood - Creatinine,Blood - Urea Nitrogen,Blood - Chloride,Blood - Bicarbonate,Blood - Hemoglobin,anchor_age,Height (Inches),Weight (Lbs),bp_systolic,bp_diastolic,99,BLD001,BLD002,BLD003,BLD004,BLD005,BLD006,BLD007,BLD008,BLD009,BLD010,CIR001,CIR002,CIR003,CIR004,CIR005,CIR006,CIR007,CIR008,CIR009,CIR010,CIR011,CIR012,CIR013,CIR014,CIR015,CIR016,CIR017,CIR018,CIR019,CIR020,CIR021,CIR022,CIR023,CIR024,CIR025,CIR026,CIR027,CIR028,CIR029,CIR030,CIR031,CIR032,CIR033,CIR034,CIR035,CIR036,CIR037,CIR038,CIR039,DEN001,DEN002,DEN003,DIG001,DIG002,DIG003,DIG004,DIG005,DIG006,DIG007,DIG008,DIG009,DIG010,DIG011,DIG012,DIG013,DIG014,DIG015,DIG016,DIG017,DIG018,DIG019,DIG020,DIG021,DIG022,DIG023,DIG024,DIG025,EAR001,EAR002,EAR003,EAR004,EAR005,EAR006,END001,END002,END003,END004,END005,END006,END007,END008,END009,END010,END011,END012,END013,END014,END015,END016,END017,EXT001,EXT002,EXT003,EXT004,EXT005,EXT006,EXT007,EXT008,EXT009,EXT010,EXT011,EXT012,EXT013,EXT014,EXT015,EXT016,EXT017,EXT018,EXT019,EXT020,EXT021,EXT022,EXT023,EXT024,EXT025,EXT026,EXT027,EXT028,EXT029,EXT030,EYE001,EYE002,EYE003,EYE004,EYE005,EYE006,EYE007,EYE008,EYE009,EYE010,EYE011,EYE012,FAC001,FAC002,FAC003,FAC004,FAC005,FAC006,FAC008,FAC009,FAC010,FAC012,FAC013,FAC014,FAC015,FAC016,FAC017,FAC019,FAC020,FAC022,FAC023,FAC024,FAC025,FAC026,FAC027,FAC028,FAC029,FAC030,GEN001,GEN002,GEN003,GEN004,GEN005,GEN006,GEN007,GEN008,GEN009,GEN010,GEN011,GEN012,GEN013,GEN014,GEN015,GEN016,GEN017,GEN018,GEN019,GEN020,GEN021,GEN022,GEN023,GEN024,GEN025,GEN026,INF001,INF002,INF003,INF004,INF005,INF006,INF007,INF008,INF009,INF010,INF011,INF012,INJ001,INJ002,INJ003,INJ004,INJ005,INJ006,INJ007,INJ008,INJ009,INJ010,INJ011,INJ012,INJ013,INJ014,INJ015,INJ016,INJ017,INJ018,INJ019,INJ020,INJ021,INJ022,INJ023,INJ024,INJ025,INJ026,INJ027,INJ028,INJ029,INJ030,INJ031,INJ032,INJ033,INJ034,INJ035,INJ036,INJ037,INJ038,INJ039,INJ040,INJ041,INJ042,INJ043,INJ044,INJ045,INJ046,INJ047,INJ048,INJ049,INJ050,INJ054,INJ055,INJ056,INJ057,INJ059,INJ060,INJ061,INJ062,INJ063,INJ064,INJ065,INJ066,INJ067,INJ069,INJ070,INJ071,INJ072,INJ073,INJ074,INJ075,INJ076,MAL001,MAL002,MAL003,MAL004,MAL005,MAL006,MAL007,MAL008,MAL009,MAL010,MBD001,MBD002,MBD003,MBD004,MBD005,MBD006,MBD007,MBD008,MBD009,MBD010,MBD011,MBD012,MBD013,MBD014,MBD017,MBD018,MBD019,MBD020,MBD021,MBD022,MBD023,MBD024,MBD025,MBD026,MBD027,MBD028,MBD032,MBD034,MUS001,MUS002,MUS003,MUS004,MUS005,MUS006,MUS007,MUS008,MUS009,MUS010,MUS011,MUS012,MUS013,MUS014,MUS015,MUS016,MUS017,MUS020,MUS021,MUS022,MUS023,MUS024,MUS025,MUS026,MUS027,MUS028,MUS029,MUS030,MUS031,MUS032,MUS033,MUS034,MUS035,MUS036,MUS037,MUS038,NEO001,NEO002,NEO003,NEO004,NEO005,NEO006,NEO007,NEO008,NEO009,NEO010,NEO011,NEO012,NEO013,NEO014,NEO015,NEO016,NEO017,NEO018,NEO019,NEO020,NEO021,NEO022,NEO023,NEO024,NEO025,NEO026,NEO027,NEO028,NEO029,NEO030,NEO031,NEO032,NEO033,NEO034,NEO035,NEO036,NEO037,NEO038,NEO039,NEO040,NEO041,NEO042,NEO043,NEO044,NEO045,NEO046,NEO047,NEO048,NEO049,NEO050,NEO051,NEO052,NEO054,NEO056,NEO057,NEO058,NEO059,NEO060,NEO061,NEO062,NEO063,NEO064,NEO065,NEO066,NEO067,NEO068,NEO069,NEO070,NEO071,NEO072,NEO073,NEO074,NEO075,NEO078,NVS001,NVS002,NVS003,NVS004,NVS005,NVS006,NVS007,NVS008,NVS009,NVS010,NVS011,NVS012,NVS013,NVS014,NVS015,NVS016,NVS017,NVS018,NVS019,NVS020,NVS021,NVS022,PNL004,PNL006,PNL009,PNL010,PNL013,PNL015,PRG002,PRG003,PRG004,PRG005,PRG006,PRG007,PRG008,PRG009,PRG010,PRG011,PRG012,PRG013,PRG014,PRG015,PRG016,PRG017,PRG018,PRG019,PRG020,PRG021,PRG022,PRG023,PRG024,PRG025,PRG026,PRG027,PRG028,PRG029,PRG030,RSP001,RSP002,RSP003,RSP004,RSP005,RSP006,RSP007,RSP008,RSP009,RSP010,RSP011,RSP012,RSP013,RSP014,RSP015,RSP016,RSP017,SKN001,SKN002,SKN003,SKN004,SKN005,SKN006,SKN007,SYM001,SYM002,SYM003,SYM004,SYM005,SYM006,SYM007,SYM008,SYM009,SYM010,SYM011,SYM012,SYM013,SYM014,SYM015,SYM016,SYM017,SYM018,99.1,A01

In [17]:
modeling_df = df_final.dropna()
modeling_df.shape

(14099, 1019)

### Train Test Split

In [18]:
from sklearn.model_selection import train_test_split

# Define features (X) and target (y)
X = modeling_df.drop(columns=['susceptible_flag'])
y = modeling_df['susceptible_flag']

# First split: Train (80%) and Test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Second split: Train (80%) → Training (60%) and Validation (20%) from X_train
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

# Print dataset sizes
print(f"Train set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}")


Train set: (8459, 1018), Validation set: (2820, 1018), Test set: (2820, 1018)


### Scaling

In [19]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler
scaler = StandardScaler()

# Fit on training data and transform both train and test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


### Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix

# Initialize and train
clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(X_train_scaled, y_train)

# Predictions
y_pred = clf.predict(X_test_scaled)

# Evaluate
print(classification_report(y_test, y_pred))
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

              precision    recall  f1-score   support

         0.0       0.61      0.56      0.59      1470
         1.0       0.56      0.61      0.58      1350

    accuracy                           0.58      2820
   macro avg       0.58      0.58      0.58      2820
weighted avg       0.59      0.58      0.58      2820

Accuracy: 0.5840
Confusion Matrix:
[[829 641]
 [532 818]]


### Random Forest

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_depth': [5, 10, 25, None],
    'min_samples_split': [2, 5, 10, 25]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_val, y_val)

# Best parameters
print(f"Best params: {grid_search.best_params_}")


Best params: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 200}


In [23]:
# Initialize and train
rf = RandomForestClassifier(n_estimators=200, max_depth = 10, min_samples_split = 5, random_state=42)
rf.fit(X_train, y_train)

# Predictions
y_pred_rf = rf.predict(X_test)

# Evaluate
print(classification_report(y_test, y_pred_rf))
print(f"Accuracy: {accuracy_score(y_test, y_pred_rf):.4f}")

# Compute confusion matrix
cm = confusion_matrix(y_test, y_pred_rf)

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

              precision    recall  f1-score   support

         0.0       0.62      0.62      0.62      1470
         1.0       0.59      0.60      0.59      1350

    accuracy                           0.61      2820
   macro avg       0.61      0.61      0.61      2820
weighted avg       0.61      0.61      0.61      2820

Accuracy: 0.6071
Confusion Matrix:
[[906 564]
 [544 806]]
